In [1]:
import tensorflow as tf
import datetime
import os
import numpy as np
import ctypes
from PIL import Image

In [2]:
#### ------------------------------------------- ####
#### ------------- LOAD THE DATA --------------- ####
#### ------------------------------------------- ####

In [3]:
MAIN_DATASET_FOLDER = "..\\dataset\\"

TEST_SUBFOLDER = os.path.join(MAIN_DATASET_FOLDER, "test")
TRAIN_SUBFOLDER = os.path.join(MAIN_DATASET_FOLDER, "train")

TEST_ESPAGNE_FOLDER = os.path.join(TEST_SUBFOLDER, "espagne")
TEST_FRANCE_FOLDER = os.path.join(TEST_SUBFOLDER, "france")
TEST_JAPON_FOLDER = os.path.join(TEST_SUBFOLDER, "japon")

TRAIN_ESPAGNE_FOLDER = os.path.join(TRAIN_SUBFOLDER, "espagne")
TRAIN_FRANCE_FOLDER = os.path.join(TRAIN_SUBFOLDER, "france")
TRAIN_JAPON_FOLDER = os.path.join(TRAIN_SUBFOLDER, "japon")


In [4]:
def fill_x_and_y_with_images_and_labels(folder, classe, dataset):

    for file in os.listdir(folder):

        file_path = os.path.join(folder, file)
        image = Image.open(file_path)
        image = image.resize((64, 64))
        if image.mode in ("RGB", "P"): image = image.convert("RGBA")
        im_arr = np.array(image).flatten()
        im_arr = im_arr / 255.0

        completed_dataset = {}
        completed_dataset["value"] = im_arr
        completed_dataset["classe"] = classe

        dataset.append(completed_dataset.copy())

In [5]:
def fill_x_and_y_with_images_and_labels(folder, classe, dataset):

    for file in os.listdir(folder):

        file_path = os.path.join(folder, file)
        image = Image.open(file_path)
        image = image.resize((64, 64))
        if image.mode in ("RGB", "P"): image = image.convert("RGBA")
        im_arr = np.array(image).flatten()
        im_arr = im_arr / 255.0

        completed_dataset = {}
        completed_dataset["value"] = im_arr
        completed_dataset["classe"] = classe

        dataset.append(completed_dataset.copy())

In [6]:
def import_dataset(folder, classe):

    dataset = []

    fill_x_and_y_with_images_and_labels(folder, classe, dataset)

    return dataset

In [7]:
x_test = []
y_test = []

x_train = []
y_train = []

class_espagne_expected_outputs = [1, 0, 0]
dataset_test_for_espagne = import_dataset(TEST_ESPAGNE_FOLDER, class_espagne_expected_outputs)
dataset_train_for_espagne = import_dataset(TRAIN_ESPAGNE_FOLDER, class_espagne_expected_outputs)

class_france_expected_outputs = [0, 1, 0]
dataset_test_for_france = import_dataset(TEST_FRANCE_FOLDER, class_france_expected_outputs)
dataset_train_for_france = import_dataset(TRAIN_FRANCE_FOLDER, class_france_expected_outputs)

class_japon_expected_outputs = [0, 0, 1]
dataset_test_for_japon = import_dataset(TEST_JAPON_FOLDER, class_japon_expected_outputs)
dataset_train_for_japon = import_dataset(TRAIN_JAPON_FOLDER, class_japon_expected_outputs)

final_dataset_test = dataset_test_for_espagne + dataset_test_for_france + dataset_test_for_japon
final_dataset_train = dataset_train_for_espagne + dataset_train_for_france + dataset_train_for_japon

In [8]:
for value in final_dataset_test:

    the_image = value["value"].tolist()
    the_classe = value["classe"]

    x_test.append(the_image)
    y_test.append(the_classe)

for value in final_dataset_train:

    the_image = value["value"].tolist()
    the_classe = value["classe"]

    x_train.append(the_image)
    y_train.append(the_classe)

dataset_inputs = x_train
dataset_outputs = y_train

initiate_dataset_inputs_size = len(dataset_inputs[0])
initiate_dataset_inputs_type = ctypes.c_float * initiate_dataset_inputs_size

classes_initiate_dataset_inputs_size = initiate_dataset_inputs_size * 2

In [9]:
inputs_result = []

for sublist in dataset_inputs:
    for item in sublist:
        inputs_result.append(item)

dataset_inputs = inputs_result

outputs_result = []

for sublist in dataset_outputs:
    for item in sublist:
        outputs_result.append(item)

dataset_outputs = outputs_result

arr = [initiate_dataset_inputs_size, classes_initiate_dataset_inputs_size, 3]
arr_size = len(arr)
arr_type = ctypes.c_int * arr_size

In [10]:
# Convert train/test to array
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [11]:
#### -------------------------------------------- ####
#### ---------------- DEFINE MODELS ------------- #### 
#### -------------------------------------------- ####

In [12]:
def create_PMC_sans_couche_cachees():
    return tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=x_train.shape[1:]), # input layer
        tf.keras.layers.Dense(3, activation='softmax') # output layer
      ])


def create_PMC_une_couche_8():
    return tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=x_train.shape[1:]), # input layer
        tf.keras.layers.Dense(8, activation='relu'), # hidden layer 1
        tf.keras.layers.Dense(3, activation='softmax') # output layer
      ])


def create_PMC_une_couche_32():
    return tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=x_train.shape[1:]), # input layer
        tf.keras.layers.Dense(32, activation='relu'), # hidden layer 1
        tf.keras.layers.Dense(3, activation='softmax') # output layer
      ])


def create_PMC_deux_couches_32_32():
    return tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=x_train.shape[1:]), # input layer
        tf.keras.layers.Dense(32, activation='relu'), # hidden layer 1
        tf.keras.layers.Dense(32, activation='relu'), # hidden layer 2
        tf.keras.layers.Dense(3, activation='softmax') # output layer
      ])


In [17]:
# Modèles (a) sans couches cachées
model_pmc_a = create_PMC_sans_couche_cachees()
model_pmc_a.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 3)                 49155     
Total params: 49,155
Trainable params: 49,155
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Modèles (b) 1 couche de 8 neurones
model_pmc_b = create_PMC_une_couche_8()
model_pmc_b.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 131080    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 27        
Total params: 131,107
Trainable params: 131,107
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Modèles (c) 1 couche de 32 neurones
model_pmc_c = create_PMC_une_couche_32()
model_pmc_c.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 32)                524320    
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 99        
Total params: 524,419
Trainable params: 524,419
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Modèles (d) 2 couches de 32 neurones
model_pmc_d = create_PMC_deux_couches_32_32()
model_pmc_d.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 32)                524320    
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 99        
Total params: 525,475
Trainable params: 525,475
Non-trainable params: 0
_________________________________________________________________


In [16]:
model_pmc_a.save("../models/keras_models/model_a.h5")
model_pmc_b.save("../models/keras_models/model_b.h5")
model_pmc_c.save("../models/keras_models/model_c.h5")
model_pmc_d.save("../models/keras_models/model_d.h5")